In [1]:
import os
from pathlib import Path
from glob import glob
import pandas as pd

!pip install xlrd

You should consider upgrading via the '/Users/g0bel1n/PycharmProjects/pythonProject/basic_env/bin/python -m pip install --upgrade pip' command.


In [2]:
current_path = Path(os.getcwd())
root = current_path.parent.absolute()
data_folder = str(root) + '/raw data/'

In [23]:
types = ["xlsx", "csv", "xls"]

os.chdir(data_folder)

tanguy = {}

print("-- XLSX FILES --")
for file_path in glob("*.xlsx"):
    file_name = file_path.split(".")[0]
    
    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        tanguy[sheet] = file.parse(sheet)
    print()
    
    
print("\n-- XLS FILES --") 
for file_path in glob("*.xls"):
    file_name = file_path.split(".")[0]
    
    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        tanguy[sheet] = file.parse(sheet)
    print()
    
print("\n-- csv FILES --")
for file_path in glob("*.csv"):
    file_name = file_path.split(".")[0]
    print(file_name)
    file_df = pd.read_csv(file_path, sep=';')
    tanguy[file_name] = file_df

print("\n-- CSV FILES --")
for file_path in glob("*.CSV"):
    file_name = file_path.split(".")[0]
    print(file_name)
    file_df = pd.read_csv(file_path, sep=';')
    tanguy[file_name] = file_df

-- XLSX FILES --
Quality Minus Junk Factors Daily sheets:
-->QMJ Factors
-->HML Devil

Volatility_data_treated sheets:
-->Sheet1

Value and Momentum Everywhere Factors Monthly sheets:
-->VME Factors

bloom_data_2 sheets:
-->CL1
-->CL2
-->CL3
-->Euribor3
-->Bund10
-->FUTURES MATURITIES

Betting Against Beta Equity Factors Daily sheets:
-->BAB Factors
-->HML Devil


-- XLS FILES --
sentiment sheets:
-->SENTIMENT


-- csv FILES --
F-F_ST_Reversal_Factor_daily

-- CSV FILES --
F-F_Research_Data_5_Factors_2x3_daily
F-F_Momentum_Factor_daily


In [25]:
tanguy['QMJ Factors']['DATE']

0       1957-07-01
1       1957-07-02
2       1957-07-03
3       1957-07-05
4       1957-07-08
           ...    
16503   2021-09-24
16504   2021-09-27
16505   2021-09-28
16506   2021-09-29
16507   2021-09-30
Name: DATE, Length: 16508, dtype: datetime64[ns]

In [27]:
import datetime

In [32]:
keys = list(tanguy.keys())
for key in keys:
    if key !='FUTURES MATURITIES':
        if type(tanguy[key].index)!=pd.DatetimeIndex:
            date_col = [col for col in ['Date','DATE','Unnamed: 0','Reported data', 'time'] if col in tanguy[key].columns]
            assert len(date_col)==1, "Date column is not unique"+ str( key)
            tanguy[key].set_index(date_col, inplace=True)
            if key == 'VME Factors':
                start_date = tanguy["VME Factors"].index.min() - pd.DateOffset(day=1)
                end_date = tanguy["VME Factors"].index.max() + pd.DateOffset(day=31)
                dates = pd.date_range(start_date, end_date, freq='D')
                tanguy["VME Factors"] = tanguy["VME Factors"].reindex(dates, method='ffill')
        tanguy[key].index.names = ["Date"]
        tanguy[key].rename(columns = lambda x: x+'_'+key,inplace=True)

In [37]:
#Now merging everything into big_base

big_base = tanguy[keys[0]]['2000':].join(tanguy[keys[1]]['2000':], how='outer')


for key in keys[2:]:
    print(key)
    if key not in ['FUTURES MATURITIES','F-F_ST_Reversal_Factor_daily', 'F-F_Research_Data_5_Factors_2x3_daily', 'F-F_Momentum_Factor_daily']: big_base = big_base.join(tanguy[key]['2000':], how='outer')

big_base.interpolate(method='time' ,axis='index', limit_area='inside', inplace=True)


['QMJ Factors', 'HML Devil', 'Sheet1', 'VME Factors', 'CL1', 'CL2', 'CL3', 'Euribor3', 'Bund10', 'FUTURES MATURITIES', 'BAB Factors', 'SENTIMENT', ]



Sheet1
VME Factors
CL1
CL2
CL3
Euribor3
Bund10
FUTURES MATURITIES
BAB Factors
SENTIMENT
F-F_ST_Reversal_Factor_daily
F-F_Research_Data_5_Factors_2x3_daily
F-F_Momentum_Factor_daily


['QMJ Factors',
 'HML Devil',
 'Sheet1',
 'VME Factors',
 'CL1',
 'CL2',
 'CL3',
 'Euribor3',
 'Bund10',
 'FUTURES MATURITIES',
 'BAB Factors',
 'SENTIMENT']

In [38]:
big_base.to_excel('giga_base.xlsx', na_rep='#N/A N/A')


In [39]:
big_base

,QMJ USA_QMJ Factors,QMJ Global_QMJ Factors,QMJ Global Ex USA_QMJ Factors,QMJ Europe_QMJ Factors,HMLD USA_HML Devil,HMLD Global_HML Devil,HMLD Global Ex USA_HML Devil,HMLD Europe_HML Devil,PX_OPEN_VIX_Sheet1,PX_HIGH_VIX_Sheet1,...,BAB Global Ex USA_BAB Factors,BAB Europe_BAB Factors,Bullish_SENTIMENT,Neutral_SENTIMENT,Bearish_SENTIMENT,Bullish 8-week Mov Avg_SENTIMENT,Bull-Bear Spread_SENTIMENT,S&P 500 Weeky High_SENTIMENT,S&P 500 Weekly Low_SENTIMENT,S&P 500 Weekly Close_SENTIMENT
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,-0.001870,0.000133,0.002284,0.004255,-0.006890,-0.005618,-0.004275,-0.004158,24.36,26.15,...,0.002757,0.005155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,0.001110,-0.001725,-0.004759,-0.000253,0.023893,0.015549,0.006767,0.013849,24.94,27.18,...,0.013752,0.023651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,0.004527,0.001617,-0.001515,-0.003912,0.009608,0.023290,0.037936,0.027106,27.98,29.00,...,0.006975,0.013357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-28,NaN,NaN,NaN,NaN,-0.008596,-0.007806,-0.006668,-0.008224,NaN,NaN,...,0.005631,0.006733,0.400359,0.309564,0.290076,0.367826,0.110284,4609.08875,4528.855,4566.605
2021-10-29,NaN,NaN,NaN,NaN,-0.012750,-0.008063,-0.001326,-0.000107,NaN,NaN,...,-0.004045,-0.001651,0.402451,0.311769,0.285779,0.367901,0.116672,4619.64750,4533.710,4581.530
2021-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.415000,0.325000,0.260000,0.368357,0.155000,4683.00000,4562.840,4671.080


In [66]:
tanguy['Sheet1']

,PX_OPEN_VIX_Sheet1_Sheet1,PX_HIGH_VIX_Sheet1_Sheet1,PX_LOW_VIX_Sheet1_Sheet1,PX_LAST_VIX_Sheet1_Sheet1,VOLUME_TOTAL_CALL_VIX_Sheet1_Sheet1,PUT_CALL_VOLUME_RATIO_CUR_DAY_VIX_Sheet1_Sheet1,VOLUME_TOTAL_PUT_VIX_Sheet1_Sheet1,TOT_OPT_VOLUME_CUR_DAY_VIX_Sheet1_Sheet1,OPEN_INT_TOTAL_CALL_VIX_Sheet1_Sheet1,OPEN_INT_TOTAL_PUT_VIX_Sheet1_Sheet1,...,12MTH_IMPVOL_100.0%MNY_DF_SX5E_Sheet1_Sheet1,12MTH_IMPVOL_105.0%MNY_DF_SX5E_Sheet1_Sheet1,12MTH_IMPVOL_110.0%MNY_DF_SX5E_Sheet1_Sheet1,12MTH_IMPVOL_90.0%MNY_DF_SX5E_Sheet1_Sheet1,12MTH_IMPVOL_95.0%MNY_DF_SX5E_Sheet1_Sheet1,24MTH_IMPVOL_100.0%MNY_DF_SX5E_Sheet1_Sheet1,24MTH_IMPVOL_105.0%MNY_DF_SX5E_Sheet1_Sheet1,24MTH_IMPVOL_110.0%MNY_DF_SX5E_Sheet1_Sheet1,24MTH_IMPVOL_90.0%MNY_DF_SX5E_Sheet1_Sheet1,24MTH_IMPVOL_95.0%MNY_DF_SX5E_Sheet1_Sheet1
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,24.36,26.15,23.98,24.21,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,24.94,27.18,24.80,27.01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,27.98,29.00,25.85,26.41,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,26.68,26.71,24.70,25.73,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,25.14,25.17,21.72,21.72,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-19,22.46,25.96,21.88,22.18,417537.0,1.2020,501893.0,919430.0,4725251.0,3133065.0,...,18.2582,16.8149,15.6974,21.7018,19.8912,17.7160,16.9146,16.2554,19.7330,18.6524
2021-05-20,22.33,23.50,20.19,20.67,208977.0,0.8944,186913.0,395890.0,4842985.0,3207980.0,...,17.7729,16.3696,15.2514,21.1484,19.3383,17.5221,16.7345,16.0898,19.4659,18.3847
2021-05-21,20.42,20.89,19.53,20.15,127120.0,1.3804,175475.0,302595.0,4884139.0,3263989.0,...,17.4594,16.1039,15.0537,20.8913,19.0643,17.3253,16.5572,15.9109,19.3004,18.2317
